<a href="https://colab.research.google.com/github/hamish-haggerty/AI-hacking/blob/master/SSL/new_opt_cancer_maintrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cancer_maintrain

> Please add a description. Don't forgot to edit the below cell!!

In [1]:
#| default_exp cancer_maintrain

Setup: Surely there is a way to get rid of having to put this cell everywhere. hmmm.

Or we can just copy paste / delete this in and out when needed. Either way, getting close to a decent workable workflow.

In [2]:
#| hide

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():
    import os
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    #os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')
    os.chdir('cancer-proj')
    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"') #does this work?
    os.system('pip install -e .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

Mounted at /content/drive


In [3]:
#| hide

from nbdev.showdoc import *

In [4]:
#| export
from fastai.vision.all import *
#| export
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *

In [5]:
def get_fnames_dict(train_dir,test_dir,class_names):
    "get dictionary of fnames"

        #files names
    fnames = get_image_files(train_dir)

    #Extract training set
    max_num =100 #maximum number of samples in each class
    count_dict = {i:0 for i in class_names}
    fnames_train = []
    for i in fnames:
        #st=extract_text(i.as_posix())
        st=label_func(i)
        
        if count_dict[st]<max_num: #no more than 100 samples per category
            fnames_train.append(i)
            count_dict[st]+=1
                    
    #We further partition fnames_train into a tune-valiation set
    count_dict2 = {i:0 for i in class_names}
    fnames_tune = []
    for i in fnames_train:
        st = label_func(i)
        if count_dict2[st] < 0.8*count_dict[st]:
            fnames_tune.append(i)
            count_dict2[st]+=1
            


    fnames_valid = get_difference(fnames_train,fnames_tune)

    fnames_test = get_difference(fnames,fnames_train) + get_image_files(test_dir)

    fnames_train = fnames_tune

    return {'fnames':fnames,'fnames_train':fnames_train,'fnames_tune':fnames_tune,
            'fnames_valid':fnames_valid,
            'fnames_test':fnames_test
            }

For now just get the data however it works. Will fix this later. But want to get `main_train` working for now

In [6]:
#| export

def get_fnames_dls_dict(train_dir,test_dir,
                        device,bs_val,bs=256,bs_tune=256,size=128,n_in=3,
                        ):

    "Wrapper that returns a dictionary with the fnames, dls etc"

    #do stuff

    class_names = get_file_lists(train_dir)['class_names']
    
    fnames_dict = get_fnames_dict(train_dir,test_dir,class_names)

    data_dict = get_data_dict(fnames_dict,train_dir,test_dir, #basic stuff needed
                  device,bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=n_in #hyperparameters
                 )

    d = {**fnames_dict,**data_dict}
    
    return d



## Load the data

In [7]:
#| hide

#Since we have cloned repository and cd'd into it (and the data itself is not stored in the
#repo) we need cd out of it, get the data, then cd back into the repo `cancer-proj`.
#This is a bit annoying, can maybe remove this later
if on_colab:
    #os.chdir('..') #assumes we are currently in cancer-proj directory
    train_dir = colab_train_dir
    test_dir = colab_test_dir
else:
    train_dir = local_train_dir
    test_dir = local_test_dir
    


#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
#bs=256
#bs=698
bs=256
bs_tune=256
size=128
bs_val=174

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

# if on_colab:
#     os.chdir('cancer-proj')

## Load aug pipelines here

In [8]:
#| hide


aug_dict = create_aug_pipelines(size=size,device=device,Augs=BYOL_Augs,TUNE_Augs=TUNE_Augs,Val_Augs=Val_Augs)
aug_pipelines = aug_dict['aug_pipelines']
aug_pipelines_tune = aug_dict['aug_pipelines_tune']
aug_pipelines_test = aug_dict['aug_pipelines_test'] 

In [9]:
#| hide
#show_bt_batch(dls=dls_train,aug=aug_pipelines,n_in=3)

In [10]:
#| hide

#show_linear_batch(dls=dls_tune,n_in=3,aug=aug_pipelines_tune,n=2,print_augs=True)

In [11]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [12]:
@patch
@delegates(Learner.fit_one_cycle)
def linear_fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tune with `freeze` for `freeze_epochs` then with `unfreeze` from `epochs` using discriminative LR"
    self.freeze()

    # for p in self.model[0].parameters():
    #     print(p.requires_grad)

    # print('should be true and false')
    
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2

    #self.fit(freeze_epochs)

    self.unfreeze()

    # for p in self.model[0].parameters():
    #     print(p.requires_grad)


    # print('should be all true..')

    self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)
    #self.fit(epochs)

In [13]:
learn.fit_one_cycle??

Object `learn.fit_one_cycle` not found.


In [14]:
# #Test:

# bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
# #model = sequential(*encoder,nn.Linear(2048,9))

# #model = LinearModel(encoder=encoder,indim=2048,outdim=9)

# model = sequential(encoder,nn.Linear(2048,9))


# # s=0
# # #for p in model.encoder.parameters():
# # for p in model[-5].parameters():
# #     s+=p.sum()

# # print(s)

# learn = Learner(dls_tune,model,splitter=default_split,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
# #learn.fine_tune(1)
# learn.linear_fine_tune(10,freeze_epochs=2,base_lr=0.004)

# # s2=0
# # #for p in model.encoder.parameters():
# # for p in model[-5].parameters():
# #     s2+=p.sum()

# # print(s2)
# # print(s==s2)


## TODO: Can probably put this in `dataloading`. No need to clog this notebook up.

In [15]:
#| hide

@patch
def lf(self:BarlowTwins, pred,*yb): return lf_bt(pred,I=self.I,lmb=self.lmb)

## main training api

In [16]:
#show_bt_batch(dls=dls_train,aug=new_aug_pipelines,n_in=3)

In [17]:
def predict_ensemble(yval,scores1,scores2):

    N=yval.shape[0]

    scores = 0.5*scores1 + 0.5*scores2

    ypred = cast(torch.argmax(scores, dim=1),TensorCategory)

    correct = (ypred == yval)#.type(torch.FloatTensor)

    #correct = (torch.argmax(ypred,dim=1) == yval).type(torch.FloatTensor)
    num_correct = correct.sum()
    accuracy = num_correct/N
    
    return ypred,accuracy.item()

In [18]:
import pickle

def save_dict_to_gdrive(d,filename):
    filename = '/content/drive/My Drive/' + filename + '.pkl'
    with open(filename, 'wb') as handle:
        pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)



def load_dict_from_gdrive(filename):
    filepath = '/content/drive/My Drive/' + filename + '.pkl'
    with open(filepath, 'rb') as handle:
        return pickle.load(handle)


In [19]:
bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)



Downloading: "https://github.com/facebookresearch/barlowtwins/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://dl.fbaipublicfiles.com/barlowtwins/ep1000_bs2048_lrw0.2_lrb0.0048_lambd0.0051/resnet50.pth" to /root/.cache/torch/hub/checkpoints/resnet50.pth


  0%|          | 0.00/90.0M [00:00<?, ?B/s]

In [20]:
def split_func(model):

    return L(list(model.encoder.parameters()),list(model.L.parameters()))


def split_encoder_projector(model):

    return L(list(model.encoder.parameters()),list(model.projector.parameters()))

In [21]:
@patch
@delegates(Learner.fit_one_cycle)
def encoder_fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tune with `freeze` for `freeze_epochs` then with `unfreeze` from `epochs` using discriminative LR"
    self.freeze()

    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    self.unfreeze()

    self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)


In [22]:
base_lr=2e-3
slice(base_lr)

slice(None, 0.002, None)

In [23]:
#test
# bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
# learn = Learner(dls_train,bt_model,splitter=split_encoder_projector, cbs=[BarlowTwins(aug_pipelines,n_in=3,lmb=1/8192,print_augs=False)])
# learn.encoder_fine_tune(1)

## Main purpose is to see if we can get the alternating scheme to work...

In [24]:
import gc

In [25]:
class LM(nn.Module):
    def __init__(self,encoder):
        super().__init__()
        self.encoder=encoder
        self.head=nn.Linear(2048,9)
        
        self.encoder.cuda()
        self.head.cuda()

    def forward(self,x):
        return self.head(self.encoder(x))

In [26]:
head = create_head(2048,9)

In [27]:
lm = LM(encoder)

In [55]:
def my_splitter(m):
    return L(sequential(*m.encoder),m.head).map(params)

split = my_splitter(lm)
print(len(split))

2


In [30]:
lm = LM(encoder)


In [31]:
def my_splitter_bt(m):
    return L(sequential(*m.encoder),m.projector).map(params)
    

In [32]:
bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
learn = Learner(dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(aug_pipelines,n_in=3,lmb=1/8192,print_augs=False)])
learn.summary()

Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main


BarlowTwinsModel (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       True      
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       True      
BatchNorm2d                               128        True      
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      True      
BatchNorm2d                 

In [33]:
learn.freeze()
learn.summary()

BarlowTwinsModel (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                 

In [34]:
learn.unfreeze()
learn.summary()

BarlowTwinsModel (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       True      
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       True      
BatchNorm2d                               128        True      
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      True      
BatchNorm2d                 

In [35]:
lm = LM(encoder)
learn = Learner(dls_tune,lm,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
learn.summary()

LM (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       True      
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       True      
BatchNorm2d                               128        True      
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      True      
BatchNorm2d                               

In [36]:
learn.freeze()
learn.summary()

LM (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                               

In [37]:
_,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
model = LM(encoder) 
#model = sequential(encoder,nn.Linear(2048,9))
learn = Learner(dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
#learn = Learner(dls_tune,model,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
learn.freeze()
learn.summary()


Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main


LM (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                               

In [38]:
model

LM(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256, 

In [40]:
def my_splitter(m):
    return L(m.encoder,m.head).map(params)

In [41]:
L(1,2) == L([1,2])

True

In [54]:
LM

__main__.LM

In [43]:
_,encoder = create_model(which_model='supervised_pretrain',ps=8192,device=device)
model = LM(encoder) 


learn = Learner(dls_tune,model,
                splitter=my_splitter,
                cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0,
                )


def hacky_fine_tune(model,epochs=2,freeze_epochs=1,lr_mult=100,
              pct_start=0.3, div=5.0
                    ):
    


    learn = Learner(dls_tune,model,
                    splitter=my_splitter,
                    cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0,
                    )
    

    learn.freeze()

    base_lr=2e-3
    learn.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99)
    learn.unfreeze()
    base_lr /= 2
    learn.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div)




#hacky_fine_tune(model,epochs=5,freeze_epochs=3,lr_mult=100)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [45]:
learn = Learner(dls_tune,encoder)
learn.freeze()
learn.summary()

Sequential (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       True      
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       True      
BatchNorm2d                               128        True      
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      True      
BatchNorm2d                       

In [61]:
#| export

class main_train:
    """Instantiate and (optionally) train the encoder. Then fine-tune the supervised model. 
    Outputs metrics on validation data"""

    def __init__(self,
                 dls_train, #used for training BT (if pretrain=True)
                 dls_tune , #used for tuning
                 dls_valid, #used to compute metrics / evaluate results. 
                 xval, #currently `predict_model` below assumes this is entire validation / test data
                 yval,
                 aug_pipelines, #the aug pipeline for self-supervised learning
                 aug_pipelines_tune, #the aug pipeline for supervised learning
                 aug_pipelines_test, #test (or valid) time augmentations 
                 initial_weights, #Which initial weights to use
                 pretrain, #Whether to fit BT
                 fit_policy, #policy for fitting BT
                 tune_fit_policy,
                 num_epochs, #number of BT fit epochs
                 numfit, #number of tune_fit epochs
                 ps=8192, #projection size
                 n_in=3, #color channels
                 indim=2048, #dimension output of encoder (2048 for resnet50)
                 outdim=9, #number of classes
                 print_report=False, #F1 metrics etc
                 print_plot=False, #ROC curve
                 ):
                 store_attr()
                 self.vocab = self.dls_valid.vocab


                 #if self.model_type == 'res_proj': test_eq(self.fit_policy,'resnet_fine_tune') #I THINK this is only viable option?

                 #self.encoder_path = f'/content/drive/My Drive/models/baselineencoder_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'
                 #self.tuned_model_path = f'/content/drive/My Drive/models/baselinefinetuned_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'

    @staticmethod
    def fit(learn,fit_policy,epochs,initial_weights):


        if fit_policy == 'encoder_fine_tune':

            #learn.encoder_fine_tune(epochs,freeze_epochs=50)
            learn.freeze()
            learn.fit_one_cycle(5)
            learn.unfreeze()
            learn.fit_one_cycle(10)

        
        elif fit_policy == 'fine_tune':
            
            if initial_weights == 'bt_pretrain':                
        
                learn.freeze()
                learn.fit_one_cycle(10, slice(0.004), pct_start=0.99)
                learn.unfreeze()
                learn.fit_one_cycle(2, slice(0.002), pct_start=0.99)
                learn.fit_one_cycle(epochs,slice(0.001,0.004))


            elif initial_weights == 'supervised_pretrain':

                #learn.fine_tune(epochs)

                learn.freeze()
                learn.fit_one_cycle(2, slice(0.002), pct_start=0.99)
                learn.unfreeze()
                learn.fit_one_cycle(2, slice(0.002), pct_start=0.99)
                learn.fit_one_cycle(epochs,slice(0.001,0.002))

              

        else: raise Exception('Fit policy not of expected form')


    def train_encoder(self):
        "create encoder and (optionally, if pretrain=True) train with BT algorith, according to fit_policy"

        try: #get existing encoder and plonk on new projector
            encoder = self.encoder

            encoder.cpu()
            bt_model = create_barlow_twins_model(encoder, hidden_size=self.ps,projection_size=self.ps,nlayers=3)
            bt_model.cuda()

        except AttributeError: #otherwise, create
            bt_model,encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=device)


        if self.pretrain: #train encoder according to fit policy

            
            learn = Learner(self.dls_train,bt_model,splitter=split_encoder_projector,cbs=[BarlowTwins(self.aug_pipelines,n_in=self.n_in,lmb=1/self.ps,print_augs=False)])
            main_train.fit(learn,self.fit_policy,epochs=self.num_epochs,initial_weights=self.initial_weights)


            
        #unfreeze_upto_last_block(bt_model.encoder)
        #print('unfroze last block...')


            #torch.save(encoder.state_dict(), self.encoder_path)

        self.encoder = bt_model.encoder


    def fine_tune(self):
        "fine tune in supervised fashion, according to tune_fit_policy"

        #encoder = pickle.loads(pickle.dumps(self.encoder)) #We might want to pretrain once and fine tune several times (varying e.g. tune augs)

        try: 
            encoder = self.encoder
        
        except AttributeError:
            _,self.encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=device)


        #model = LinearModel(encoder=self.encoder,indim=self.indim,outdim=self.outdim) #create 'linear' model (encoder + linear head)
        #model = sequential(self.encoder,nn.Linear(2048,9))
        model = LM(self.encoder)

        #model = sequential(self.encoder,model)  


        learn = Learner(self.dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        #learn = Learner(self.dls_tune,model,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)


        main_train.fit(learn,fit_policy=self.tune_fit_policy,epochs=self.numfit,initial_weights=self.initial_weights) #fine tuning (don't confuse this with fit policy!)
        #hacky_fine_tune(model,epochs=self.numfit)



        scores,preds, acc = predict_model(self.xval,self.yval,model=model,aug_pipelines_test=self.aug_pipelines_test,numavg=3)

        #metrics dict will have f1 score, auc etc etc
        metrics = classification_report_wrapper(preds, self.yval, self.vocab, print_report=self.print_report)
        metrics['acc'] = acc
        auc_dict = plot_roc(self.yval,preds,self.vocab,print_plot=self.print_plot)
        metrics['auc_dict'] = auc_dict

        metrics['scores'] = scores
        metrics['preds'] = preds

        metrics['xval'] = self.yval
        metrics['yval'] = self.yval
        
        #torch.save(model.state_dict(), self.tuned_model_path)
        return metrics #

    def __call__(self):

        self.train_encoder() #train (or extract) the encoder
        metrics = self.fine_tune()
        
        return metrics

if __name__ == '__main__' and on_colab:

    initial_weights = 'supervised_pretrain'
    pretrain=True
    numfit=50
    num_epochs=20

    num=3
    avg_acc=0

    main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
            aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
            initial_weights=initial_weights,pretrain=pretrain,fit_policy='encoder_fine_tune',tune_fit_policy='fine_tune',num_epochs=num_epochs,numfit=numfit,
            print_report=True,
            ps=8192
                    )
    
    metrics = main.fine_tune()

    # metrics = main.train_encoder()
    # metrics = main.fine_tune()
    # metrics = main.train_encoder()
    # metrics = main.fine_tune()

    # torch.cuda.empty_cache()
    # gc.collect()

    # main.numfit=50
    # metrics = main.fine_tune()

    #metrics = main.fine_tune()
    #metrics = main.fine_tune()

    #avg_acc += metrics['acc']

    #print(metrics['acc'])

    print(avg_acc/num)
    # metrics = main()
    # metrics=main()


epoch,train_loss,valid_loss,time
0,2.214560,None,00:06
1,2.197483,None,00:06


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,2.115308,None,00:06
1,2.091568,None,00:06


epoch,train_loss,valid_loss,time
0,1.922571,None,00:06
1,1.918953,None,00:06
2,1.882776,None,00:06
3,1.832486,None,00:06
4,1.766564,None,00:06
5,1.682367,None,00:06
6,1.580390,None,00:06
7,1.466143,None,00:06
8,1.357134,None,00:06
9,1.249773,None,00:06


                            precision    recall  f1-score   support

         actinic keratosis       0.50      0.75      0.60        20
      basal cell carcinoma       0.67      0.60      0.63        20
            dermatofibroma       0.94      0.84      0.89        19
                  melanoma       0.38      0.30      0.33        20
                     nevus       0.53      0.45      0.49        20
pigmented benign keratosis       0.76      0.65      0.70        20
      seborrheic keratosis       0.32      0.40      0.35        15
   squamous cell carcinoma       0.58      0.55      0.56        20
           vascular lesion       0.95      1.00      0.98        20

                  accuracy                           0.62       174
                 macro avg       0.62      0.62      0.62       174
              weighted avg       0.63      0.62      0.62       174

0.0
